Import libraries.


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import random
from sklearn import metrics
%matplotlib inline

Import classes.


In [ ]:
from app.utils.HistogramEqualization import HistogramEqualization
from app.utils.ResNet34Model import ResNet34Model
from app.utils.VGG16Model import VGG16Model
from app.utils.EfficientNetModel import EfficientNetModel
from app.utils.ImageDataLoader import ImageDataLoader
from app.utils.ImageProcessor import ImageProcessor
from app.utils.DataExplorer import DataExplorer
from app.utils.DatasetStatistics import DatasetStatistics
from app.utils.DuplicateDetector import DuplicateDetector
from app.utils.OversampledDataset import OversampledDataset
from app.utils.BatchVisualizer import BatchVisualizer
from app.utils.Trainer import Trainer

In [ ]:
%run DatasetDivider.ipynb

Prepare constant variables.


In [ ]:
TRAIN_DIR = "./new_data/Training"
VAL_DIR = "./new_data/Validation"
TEST_DIR = "./new_data/Testing"

BATCH_SIZE = 64
IMAGE_SIZE = (224, 224)
NUMBER_OF_CLASSES = 4

FINAL_EPOCHS = 50
PATIENCE = 10

Definition of function setting same seed to maintain reproducibility.


In [ ]:
def set_seed(seed=42):
    random.seed(seed)  # Python random
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (used)
    torch.cuda.manual_seed_all(seed)  # PyTorch all GPUs
    torch.backends.cudnn.deterministic = True  # CUDA deterministic operations
    torch.backends.cudnn.benchmark = False  # Turn off autotuning for reproducity

Check if accelerator is available. If not, use cpu instead.


In [ ]:
device = (
    torch.accelerator.current_accelerator().type
    if torch.accelerator.is_available()
    else "cpu"
)
print(f"Using {device} device")
if device == "cuda":
    print(f"Accelerator name: {torch.cuda.get_device_name(device)}")
    torch.cuda.empty_cache()

**Run following cell only if new dataset division is necessary**.<br>
Otherwise, skip this step.


In [ ]:
divider = DatasetDivider()
split_data = divider.run()

Load images from specified train, validation and test directories. Validation directory is optional.


In [ ]:
loader = ImageDataLoader(TRAIN_DIR, VAL_DIR, TEST_DIR)
all_files = loader.load_all_images()

print(f"Successfully loaded {len(all_files)} images")

Check loaded data for any instances of duplicate. Check class distribution.


In [ ]:
duplicate_detector = DuplicateDetector(all_files)
duplicate_detector.detect_duplicates()

if len(duplicate_detector.duplicates) > 0:  # If duplicate files are present
    duplicate_detector.remove_duplicates_from_disk()  # Removing duplicates entirely from disk
    all_files = duplicate_detector.get_unique_files()  # Cleaning list with file paths

In [ ]:
loader.print_dataset_class_count()

Convert loaded images to grayscale, apply histogram equalization.<br>
Display batch of 32 images.


In [ ]:
processor = ImageProcessor(all_files)

processor.load_grayscale_images(equalize=True)
processor.display_image_grid(batch_size=32, figsize=(18, 9), images_per_row=8)

Get 1st, middle and last images' histograms.


In [ ]:
explorator = DataExplorer(all_files)
explorator.retrieve_sample_of_images(
    [0, len(all_files) // 2, -1], equalize=True
)  # First, middle and last image

explorator.plot_histogram()

Compute mean and standard deviation values based on provided dataset. Default values for ImageNet set are inappropriate due to gray-scale MRI format.


In [ ]:
stats = DatasetStatistics(processor.gray_images)

stats.compute_stats()
stats.print_stats()

MEAN, STD = stats.get_normalized_values()

Prepare transformations, including data augmentation for training set. `test_transform` will be applied to validation set as well.


In [ ]:
train_transform = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        HistogramEqualization(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(12),
        transforms.ColorJitter(brightness=0.15, contrast=0.15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[MEAN] * 3, std=[STD] * 3),
    ]
)

In [ ]:
test_transform = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        HistogramEqualization(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[MEAN] * 3, std=[STD] * 3),
    ]
)

Use generic data loader and apply transformations.


In [ ]:
trainset = OversampledDataset(TRAIN_DIR, transform=train_transform)
valset = datasets.ImageFolder(VAL_DIR, transform=test_transform)
testset = datasets.ImageFolder(TEST_DIR, transform=test_transform)

print()
trainset.print_class_distribution()

print()
print(f"Training samples: {len(trainset)}")
print(f"Validation samples: {len(valset)}")
print(f"Test samples: {len(testset)}")
print(f"Classes: {trainset.classes}")

Seed the generator passed to `DataLoader` for reproducibility. Shuffle tran samples to avoid samples being remembered instead of generalized.


In [ ]:
g = torch.Generator()
g.manual_seed(42)

train_dl = DataLoader(trainset, BATCH_SIZE, shuffle=True, num_workers=0, generator=g)
val_dl = DataLoader(valset, BATCH_SIZE, shuffle=False, num_workers=0)
test_dl = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
visualizer = BatchVisualizer(trainset.classes, mean=[MEAN] * 3, std=[STD] * 3)

visualizer.visualize_batch(train_dl)
visualizer.visualize_classes(test_dl, 6)

Configure models, optimizers and learning rates to later on run experiments and choose the best combinations of these parameters.


In [ ]:
models_config = {
    "VGG-16": VGG16Model,
    "ResNet34": ResNet34Model,
    "EfficientNet_B0": EfficientNetModel,
}

optimizers_config = {
    "sgd": lambda params, lr: optim.SGD(params, lr=lr),
    "sgd_momentum": lambda params, lr: optim.SGD(params, lr=lr, momentum=0.9),
    "adam": lambda params, lr: optim.Adam(params, lr=lr),
}

LEARNING_RATES = [0.01, 0.001, 0.0001, 0.00001]
EPOCHS = 5

In [ ]:
def run_experiment(
    model_class, optimizer_fn, lr, train_dl, val_dl, device, epochs=EPOCHS
):
    set_seed(42)

    model = model_class(NUMBER_OF_CLASSES)
    trainer = Trainer(model, device)
    optimizer = optimizer_fn(model.parameters(), lr)

    history = trainer.fit(train_dl, val_dl, optimizer, epochs)

    return {
        "last_epoch_val_acc": history["val_acc"][-1],
        "best_val_acc": max(history["val_acc"]),
        "last_epoch_val_loss": history["val_loss"][-1],
        "history": history,
    }


def run_all_experiments(train_dl, val_dl, device):
    res = []

    total = len(models_config) * len(optimizers_config) * len(LEARNING_RATES)
    current = 0

    for model_name, model_class in models_config.items():
        for optim_name, optim_fn in optimizers_config.items():
            for lr in LEARNING_RATES:
                current += 1
                print(f"\n[{current}/{total}] {model_name} + {optim_name} + lr={lr}")

                result = run_experiment(
                    model_class, optim_fn, lr, train_dl, val_dl, device
                )

                res.append(
                    {
                        "model": model_name,
                        "optimizer": optim_name,
                        "learning rate": lr,
                        **result,
                    }
                )
                print(
                    f"Last epoch's validation accuracy: {result['last_epoch_val_acc']:.4f}"
                )

    return res

In [ ]:
results = run_all_experiments(train_dl, val_dl, device)

Amongst VGG-16 model combinations, Adam optimizer + lr=0.0001 turned out to be the best in regard of accuracy.


Train and test the model, measure its performance.


In [ ]:
set_seed(42)

vgg16_model = VGG16Model(NUMBER_OF_CLASSES)
optimizer_vgg16 = optim.Adam(vgg16_model.parameters(), lr=0.0001)

trainer_vgg16 = Trainer(vgg16_model, device)
vgg16_history = trainer_vgg16.fit(
    train_dl, val_dl, optimizer_vgg16, FINAL_EPOCHS, PATIENCE
)
vgg16_acc, vgg16_preds, vgg16_labels = trainer_vgg16.test(test_dl)

In [ ]:
print("#" * 20 + " VGG-16 CLASSIFICATION STATISTICS " + "#" * 20)
print()

print(
    metrics.classification_report(
        vgg16_labels, vgg16_preds, target_names=testset.classes
    )
)

Amongst ResNet-34 model combinations, stochastic gradient descent optimizer + lr=0.001 turned out to be the best in regard of accuracy.


Train and test the model, measure its performance.


In [ ]:
set_seed(42)

resnet34_model = ResNet34Model()
optimizer_resnet = optim.SGD(resnet34_model.parameters(), lr=0.001, momentum=0.9)

trainer_resnet34 = Trainer(resnet34_model, device)
resnet34_history = trainer_resnet34.fit(
    train_dl, val_dl, optimizer_resnet, FINAL_EPOCHS, PATIENCE
)
resnet34_acc, resnet34_preds, resnet34_labels = trainer_resnet34.test(test_dl)

In [ ]:
print("#" * 20 + " RESNET34 CLASSIFICATION STATISTICS " + "#" * 20)
print()

print(
    metrics.classification_report(
        resnet34_labels, resnet34_preds, target_names=testset.classes
    )
)

Amongst EfficientNet B0 weights model combinations, Adam optimizer + lr=0.001 turned out to be the best in regard of accuracy.


Train and test the model, measure its performance.


In [ ]:
set_seed(42)

efficientnet_model = EfficientNetModel()
optimizer_efficientnet = optim.Adam(efficientnet_model.parameters(), lr=0.001)

trainer_efficientnet = Trainer(efficientnet_model, device)
efficientnet_history = trainer_efficientnet.fit(
    train_dl, val_dl, optimizer_efficientnet, FINAL_EPOCHS, PATIENCE
)
efficientnet_acc, efficientnet_preds, efficientnet_labels = trainer_efficientnet.test(
    test_dl
)

In [ ]:
print("#" * 20 + " EFFICIENTNET CLASSIFICATION STATISTICS " + "#" * 20)
print()

print(
    metrics.classification_report(
        efficientnet_labels, efficientnet_preds, target_names=testset.classes
    )
)

Plotting training & validation set accuracy and loss functions definitions


In [ ]:
def plot_train_val_accuracy(train_acc, val_acc, title=None, ax=None):
    if ax is None:
        _, ax = plt.subplots()

    ax.plot(train_acc, "r--", label="Training accuracy")
    ax.plot(val_acc, "b-", label="Validation accuracy")

    ax.set_xlabel("Epoch")
    ax.set_ylabel("Accuracy")
    if title is not None:
        ax.set_title(title, fontsize=10)

    ax.grid()
    ax.legend()


def plot_train_val_loss(train_loss, val_loss, title=None, ax=None):
    if ax is None:
        _, ax = plt.subplots()

    ax.plot(train_loss, "g--", label="Training loss")
    ax.plot(val_loss, "m-", label="Validation loss")

    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    if title is not None:
        ax.set_title(title, fontsize=10)

    ax.grid()
    ax.legend()

Plot VGG-16, ResNet-34 and EfficientNet train & validation set accuracy against each other.


In [ ]:
_, axs = plt.subplots(ncols=3, figsize=(15, 5))
plot_train_val_accuracy(
    vgg16_history["train_acc"], vgg16_history["val_acc"], "VGG-16 model", axs[0]
)

plot_train_val_accuracy(
    resnet34_history["train_acc"], resnet34_history["val_acc"], "ResNet34 model", axs[1]
)

plot_train_val_accuracy(
    efficientnet_history["train_acc"],
    efficientnet_history["val_acc"],
    "EfficientNet model",
    axs[2],
)

plt.suptitle("Models training accuracy vs validation accuracy", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

Plot VGG-16, ResNet-34 and EfficientNet train & validation set loss against each other.


In [ ]:
_, axs = plt.subplots(ncols=3, figsize=(15, 5))
plot_train_val_loss(
    vgg16_history["train_loss"], vgg16_history["val_loss"], "VGG-16 model", axs[0]
)

plot_train_val_loss(
    resnet34_history["train_loss"],
    resnet34_history["val_loss"],
    "ResNet34 model",
    axs[1],
)

plot_train_val_loss(
    efficientnet_history["train_loss"],
    efficientnet_history["val_loss"],
    "EfficientNet model",
    axs[2],
)

plt.suptitle("Models training loss vs validation loss", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

Aggregate the results to compare models. Plot confusion matrix for each one.


In [ ]:
results = [
    [vgg16_labels, vgg16_preds],
    [resnet34_labels, resnet34_preds],
    [efficientnet_labels, efficientnet_preds],
]

_, axs = plt.subplots(ncols=3, figsize=(28, 8))
i = 0

for model in models_config:
    cm = metrics.confusion_matrix(results[i][0], results[i][1])
    cm_display = metrics.ConfusionMatrixDisplay(cm, display_labels=testset.class_to_idx)
    cm_display.plot(cmap="Blues", ax=axs[i], text_kw={"fontsize": 14})

    axs[i].set_title(f"{model} confusion matrix", y=1.02, fontsize=18)

    axs[i].tick_params(axis="x", rotation=45, labelsize=12)
    axs[i].set_xticklabels(axs[i].get_xticklabels(), ha="right")

    axs[i].tick_params(axis="y", labelsize=12)

    axs[i].set_xlabel("Predicted label", fontsize=14)
    axs[i].set_ylabel("True label", fontsize=14)

    i += 1

plt.suptitle("Confusion matrix for each model", fontsize="22", y=1.05)
plt.tight_layout()
plt.show()

Additional experiments showed ResNet's model superiority over VGG-16 and EfficienNet models. Final setting:<br>
**ResNet34 + sgd_momentum=0.9 + lr=0.001, collorJitter settings at .15 and histogram equalization**<br><br>
Export the model to .pth file.


In [ ]:
checkpoint = {
    "weights": trainer_resnet34.model.state_dict(),
    "mean": MEAN,
    "std": STD,
    "image_size": IMAGE_SIZE,
    "hist_eq": True,
}
# torch.save(checkpoint, "./config/resnet34.pth")